# First Analysis of the Experiment Data

In [1]:
import numpy as np
import pandas as pd
import collections, os, sys, random, time, pickle, dill, copy, re
from IPython.display import clear_output
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from tf.fabric import Fabric
from tf.extra.bhsa import Bhsa
from project_code.experiments2 import Experiment
from project_code.semspace import SemSpace

bhsa_data_paths=['~/github/etcbc/bhsa/tf/c',
                 '~/github/verb_semantics/project_code/lingo/heads/tf/c',
                 '~/github/verb_semantics/project_code/sdbh']

TF = Fabric(bhsa_data_paths)
tf_api = TF.load('''
                function lex lex_utf8 vs language
                pdp freq_lex gloss domain ls
                mother rela typ sp st code txt instruction
                heads prep_obj 
                prs prs_gn prs_nu prs_ps
                sem_domain sem_domain_code
              ''', silent=True)

tf_api.makeAvailableIn(globals())
B = Bhsa(api=tf_api, name='', version='c')

This is Text-Fabric 4.4.1
Api reference : https://dans-labs.github.io/text-fabric/Api/General/
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

118 features found and 0 ignored


**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="{provenance of this corpus}">BHSA</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/c/0_home.html" title="{CORPUS} feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Bhsa/" title="BHSA API documentation">BHSA API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 4.4.1</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/#search-templates" title="Search Templates Introduction and Reference">Search Reference</a>

In [2]:
print('Loading cached experiments...')
with open('/Users/cody/Documents/experiments.dill', 'rb') as infile:
    experiments = dill.load(infile)
    print(f'{len(experiments)} experiments loaded.')

Loading cached experiments...
50 experiments loaded.


In [3]:
print('Loading semantic space data: adjusting counts with pmi, preparing plotter functions, etc...')

spaces = dict((exp_name, SemSpace(exp, info=False)) for exp_name, exp in experiments.items())

print(f'{len(spaces)} experiments successfully loaded into semantic spaces.')

Loading semantic space data: adjusting counts with pmi, preparing plotter functions, etc...
50 experiments successfully loaded into semantic spaces.


## Experiments and their Shapes

In [4]:
for shape, exp in sorted((experiments[exp].data.shape, exp) for exp in experiments):
    print(f'{exp}:\t{shape}')

vi_subj_animacy:	(2, 180)
vi_objc_pa:	(2, 714)
vi_adj+_pa:	(2, 734)
vi_cmpl_pa:	(2, 734)
vi_coad_pa:	(2, 734)
vf_obj_pa:	(3, 694)
vd_domain_simple:	(3, 704)
vf_cmpl_pa:	(4, 725)
vi_allarg_pa:	(4, 786)
vi_objc_animacy:	(5, 173)
vf_adju_pa:	(7, 733)
vf_obj_animacy:	(8, 127)
vf_coad_pa:	(8, 734)
vg_tense:	(8, 734)
vi_cmpl_animacy:	(39, 174)
vf_argAll_pa:	(43, 703)
vi_adj+_animacy:	(46, 108)
vi_coad_animacy:	(51, 241)
vd_domain_embed:	(73, 646)
vf_cmpl_animacy:	(88, 158)
vi_allarg_animacy:	(92, 370)
vf_adju_animacy:	(96, 78)
vf_coad_animacy:	(200, 192)
vi_subj_domain:	(247, 231)
vd_par_lex:	(305, 365)
vf_argAll_animacy:	(378, 207)
vi_objc_domain:	(448, 245)
vf_obj_domain:	(584, 213)
vi_cmpl_domain:	(1033, 223)
vf_cmpl_domain:	(1128, 207)
vi_adj+_domain:	(1219, 217)
vf_adju_domain:	(1575, 180)
vi_coad_domain:	(1735, 386)
vi_subj_lex:	(1959, 290)
vi_objc_lex:	(2251, 305)
vi_allarg_domain:	(2902, 527)
vf_obj_lex:	(3012, 274)
vf_coad_domain:	(3055, 301)
vi_adj+_lex:	(3478, 295)
vf_cmpl_lex:	(4

## Deriving a Verb Specimen Set

Which verbs will comprise this study? This depends on which verbs are accounted for in the dataset. The presence/absence (pa) experiments count the most basic features, namely, the simple presence or absence of a given verbal argument: objects, complements, adjuncts. I will first try to derive a specimen set by taking the intersection of all the pa experiments.

In [5]:
specimens = set(experiments['vi_objc_pa'].data.columns)
print(f'Starting with specimens count of {len(specimens)}')

for exp_name, experiment in experiments.items():
    
    if exp_name == 'vi_objc_pa' or not re.match('.*_pa', exp_name): # skip first pa experiments or non-pa exps.
        continue
        
    exp_lexemes = set(experiment.data.columns)
    specimens = exp_lexemes & specimens
    
print(f'Complete with specimen count of {len(specimens)}')

Starting with specimens count of 714
Complete with specimen count of 297


### Count Lexemes without Stems

All verbs have their stem appended. How many plain lexemes are in the specimen set?

In [6]:
plain_lexs = set(lex.split('.')[0] for lex in specimens)
lex_nodes = [(next(l for l in F.otype.s('lex') if F.lex.v(l) == lex),) for lex in plain_lexs]

print(f'Number of plain lexemes accounted for: {len(plain_lexs)}\n')
B.show(lex_nodes)

Number of plain lexemes accounted for: 243




##### Passage 1



##### Passage 2



##### Passage 3



##### Passage 4


...


##### Passage 97



##### Passage 98



##### Passage 99



##### Passage 100


**120 more results skipped** because we show a maximum of 100 results at a time

We have a good and diverse dataset here. Let's see what the makeup of stems are.

### Stem Counts

In [7]:
stem_counts = collections.Counter(lex.split('.')[1] for lex in specimens)

stem_counts.most_common()

[('qal', 167),
 ('hif', 59),
 ('piel', 41),
 ('nif', 24),
 ('hit', 4),
 ('hsht', 1),
 ('hof', 1)]

## Coverage of Specimens throughout the Experiments

There are 297 verb specimens present with 243 plain lexical forms. Note that many of the experiments have different numbers of specimens present, due to the various requirements per experiment. We intend to use as much data as is present to inform the verb clustering. But are there some verbs whose coverage throughout all 50 experiments is especially lacking?

Below we count per lexeme how many different experiments it occurs in. The number itself is then counted.

In [8]:
spread_count = collections.Counter()
accounted_exps = set()

for lex in specimens:
    count = 0
    for exp_name, exp in experiments.items():
        if lex in exp.data.columns:
            count += 1
            accounted_exps.add(exp_name)
    spread_count[count] += 1
    
spread_count.most_common()

[(50, 26),
 (48, 14),
 (17, 14),
 (44, 13),
 (25, 13),
 (18, 13),
 (19, 12),
 (41, 12),
 (40, 11),
 (21, 11),
 (42, 10),
 (49, 10),
 (32, 10),
 (29, 9),
 (34, 8),
 (23, 8),
 (37, 8),
 (39, 8),
 (22, 7),
 (36, 7),
 (46, 7),
 (33, 7),
 (30, 6),
 (31, 6),
 (24, 6),
 (27, 5),
 (26, 5),
 (43, 5),
 (47, 5),
 (45, 4),
 (38, 4),
 (20, 4),
 (35, 3),
 (15, 2),
 (28, 2),
 (16, 2)]

What is the lowest accounted-for lexemes and how many of them are there?

In [9]:
min(spread_count.most_common())

(15, 2)

In [10]:
sum([amount for count, amount in spread_count.most_common() if count<20])

43

In [11]:
43/297 # ratio of total specimens within 20 or less experiments

0.1447811447811448

Only 2 lexemes are accounted for in only 15 of the experiments. 43 of the 297 verb specimens (14%) have less than 20 applicable experiments. These are acceptable amounts. Verbs that do not have an attested object, for instance, will not appear in 3 of 4 object experiments. But they *will* occur in the presence/absence object experiment. So it can be expected that certain verbs will be less accounted-for in various experiments.

## First Similarity Analysis

For each specimen, calculate its similarity with all other specimens pairwise based on all experiments which they have in common. The similarity scores will be averaged across all of the shared experiments. All raw counts have been adjusted using the pointwise mutual information (pmi) score. 

In [20]:
sim_matrix = pd.DataFrame(np.zeros(shape=(len(specimens), len(specimens))), columns=specimens, index=specimens) # for pairwise similarities
common_matrix = pd.DataFrame(np.zeros(shape=(len(specimens), len(specimens))), columns=specimens, index=specimens) # for counting number of common experiments

for lex in specimens:
    for space_name, space in spaces.items():
        
        if lex not in space.sim_pmi_nogloss.columns:
            continue
        
        sim_matrix[lex] = sim_matrix[lex].add(space.sim_pmi_nogloss[lex], fill_value=0)
        common_matrix[lex] += 1
        
sim = sim_matrix / common_matrix

In [61]:
list(sim.columns)

['RWM[.hif',
 'FXQ[.qal',
 'XJH[.piel',
 '<BR[.hif',
 '>HB[.qal',
 'QN>[.piel',
 'PLL[.hit',
 'JRH[.hif',
 'JRD[.qal',
 'YDQ[.qal',
 '>RB[.qal',
 'FMX[.qal',
 'XZQ[.hif',
 'XCK[.qal',
 'NWX[.qal',
 'FKL[.hif',
 'FXQ[.piel',
 'PRX[.qal',
 'XLL[.piel',
 'JD<[.hif',
 'BWC[.qal',
 'BQ<[.qal',
 'HLK[.hit',
 'CLX[.qal',
 'SWR[.qal',
 'MWT[.qal',
 'JYT[.hif',
 'R>H[.qal',
 'ZKR[.qal',
 'MY>[.qal',
 'XKM[.qal',
 'XLP[.qal',
 'LQX[.qal',
 'NF>[.qal',
 'XMD[.qal',
 'YWH[.piel',
 'FMX[.piel',
 'XWS[.qal',
 'NG<[.qal',
 'GDL[.piel',
 'XPY[.qal',
 'QDM[.piel',
 'GWR==[.qal',
 'HLK[.piel',
 'CQL[.qal',
 '>BH[.qal',
 'CNH[.qal',
 'MLV[.piel',
 'GDL[.hif',
 '<YR[.qal',
 'JD<[.qal',
 'LMD[.piel',
 'RBH[.hif',
 'BNH[.qal',
 'SBB[.qal',
 'DRK[.hif',
 'XDL[.qal',
 'CGH[.qal',
 'STR[.hif',
 'SGR[.qal',
 'YRR[.qal',
 'CWB[.qal',
 'JHB[.qal',
 'NYR[.qal',
 'JY>[.qal',
 'GJL[.qal',
 'HDP[.qal',
 'JBC[.hif',
 'NFG[.hif',
 'JRD[.hif',
 '>WR[.hif',
 'XRC=[.qal',
 'BJN[.qal',
 'PYH[.qal',
 'BQ<[.nif',
 'RJB[.qal'

In [76]:
sim['PYH[.qal']['PTX[.qal']

0.033968472930099095

In [84]:
sim['HJH[.qal'].sort_values(ascending=False)

HJH[.qal      1.000000
NTN[.qal      0.392858
<FH[.qal      0.375346
BW>[.qal      0.374886
JCB[.qal      0.352519
JY>[.qal      0.346496
CWB[.qal      0.342169
<LH[.qal      0.341492
NPL[.qal      0.329719
>MR[.qal      0.328196
HLK[.qal      0.327721
<MD[.qal      0.322628
<BR[.qal      0.319955
QWM[.qal      0.319679
DBR[.piel     0.311541
MWT[.qal      0.305919
QR>[.qal      0.305039
R>H[.qal      0.293011
MLK[.qal      0.292025
CM<[.qal      0.289863
LQX[.qal      0.289727
FJM[.qal      0.285303
JRD[.qal      0.284160
CKB[.qal      0.282453
CLX[.qal      0.278645
BW>[.hif      0.276342
>KL[.qal      0.276220
NF>[.qal      0.270708
BNH[.qal      0.262735
YWH[.piel     0.261860
                ...   
<LZ[.qal      0.007063
XLH[.piel     0.007045
CMD[.nif      0.006999
SGR[.qal      0.006894
NDD[.qal      0.006666
YPN[.qal      0.006480
S<D[.qal      0.006359
>MY[.piel     0.006278
PQX[.qal      0.006038
VBL[.qal      0.005816
FB<[.hif      0.005576
>TH[.qal      0.005561
PTX[.nif   